In [2]:
import json
import codecs
import csv
import pandas as pd

NEO_VER_CSV = 'data/out/neo_ver.csv'
NEO_OUT_JSON = 'data/input/records.json'
CROSS_CSV = 'ne4out.csv'
# df : pd.read_csv(NEO_VER_CSV)

# def get_csv_data(file_path):
#     with open(file_path) as f:
#         data : csv.reader(f)
#     return data

# jdata = json.load(codecs.open(NEO_OUT_JSON, 'r', 'utf-8-sig'))

In [ ]:
# Single example of getting a dataframe with packages that have same repo url

dfs = []
df2 = df[df['pkg_name'] == 'glib']
df2 = df2[~df2['url'].isnull()]
pkgmans = df2['pkgman'].unique()
for pkgman in pkgmans:
    dfs.append(df2[df2['pkgman'] == pkgman]) # Only drops repo duplicated because duplicate e.g. NaN in homepage would drop two different repos

master_df = pd.concat(dfs)
df_repo = master_df[master_df.duplicated(subset='url', keep=False)]
df_repo

In [31]:

rhdf = master_df[master_df.duplicated(subset='url', keep=False)]
rhdf = rhdf[~rhdf['url'].isnull()]
rhdf

,:ID,pkg_name,pkgman,version,:LABEL,url
102530,version62888cargo,glib,cargo,0.3.0,Version,https://github.com/gtk-rs/gtk-rs-core
102531,version157792cargo,glib,cargo,0.8.0,Version,https://github.com/gtk-rs/gtk-rs-core
102532,version203315cargo,glib,cargo,0.9.1,Version,https://github.com/gtk-rs/gtk-rs-core
102533,version17029cargo,glib,cargo,0.0.5,Version,https://github.com/gtk-rs/gtk-rs-core
102534,version416599cargo,glib,cargo,0.14.4,Version,https://github.com/gtk-rs/gtk-rs-core
...,...,...,...,...,...,...
17428076,version16696003npm,glib,npm,1.0.1,Version,git://github.com/bleupen/hapi-logger.git
17428077,version16696004npm,glib,npm,1.0.2,Version,git://github.com/bleupen/hapi-logger.git
17428078,version16696005npm,glib,npm,1.1.0,Version,git://github.com/bleupen/hapi-logger.git
17428079,version16696006npm,glib,npm,1.2.0,Version,git://github.com/bleupen/hapi-logger.git


In [12]:
hrdf = master_df[master_df["homepage"].isin(master_df["repo"]) + master_df["repo"].isin(master_df["homepage"])]
hrdf

NameError: name 'master_df' is not defined

In [ ]:
pd.concat([df_repo, rhdf, hrdf]).drop_duplicates()

In [38]:
import math

# Example that iterates the records.json
result = []
TOTAL = len(jdata)
CURRENT = 0
PROGRESS = 0

for r in jdata:

    # Get name pkg and create initial list for dataframes
    name = r["name"]
    frames = []

    CURRENT += 1

    if PROGRESS < math.ceil((CURRENT/TOTAL)*100):
        PROGRESS = math.ceil((CURRENT/TOTAL)*100)
        print(f"Processing {PROGRESS}%  | {CURRENT}/{TOTAL}")

    # For each pkg(name), iterate the pkg managers 
    for pkgman in r["pkgmans"]:

        # Get df from NEO_VER for that pkg and remove rows with NaN in 'repo'
        ndf = df[df['pkg_name'] == name]
        ndf = ndf[~ndf['url'].isnull()]

        # Get df without duplicates and append to dataframes list
        frames.append(ndf[ndf['pkgman'] == pkgman].drop_duplicates('url'))

    # Concat all frames and keep only duplicated on repo field. Duplicated would be those who appear < 1
    mdf = pd.concat(frames)
    udf = mdf[mdf.duplicated(subset='url', keep=False)]
    mdf = udf.drop_duplicates()

    # Only add if merged dataframe is not empty
    if not mdf.empty:
        result.append(mdf)
        
fin = pd.concat(result)
fin.to_csv("out/neo4jout3.csv", encoding='utf-8', index=False)

Processing 13%  | 1/8
Processing 25%  | 2/8
Processing 38%  | 3/8
Processing 50%  | 4/8
Processing 63%  | 5/8
Processing 75%  | 6/8
Processing 88%  | 7/8
Processing 100%  | 8/8


In [23]:
CROSS_CSV = 'out/ne4out3.csv'
df = pd.read_csv(CROSS_CSV)
df = df[df.duplicated(subset=['pkg_name'], keep=False)] # remove packages that occur once, but is a hit because repo and homepage are the same

df = df[df.duplicated(subset=['url'], keep=False)]



#df 
df.to_csv('out/non_unique_repo.csv', encoding='utf-8', index=False)


In [41]:
nu_df = pd.read_csv('out/non_unique_repo.csv')
un_df = nu_df['pkg_name'].unique()
len(un_df) # 993 unique pkgs


853

In [21]:
gp_df = nu_df.groupby(['pkgman'])
gp_df.size().sort_values(ascending=False)
# freebsd12     381
# conan         341
# vcpkg         337
# openbsd69     271
# netbsd9       180
# homebrew       88
# chromebrew     86
# npm            27
# cargo          26
# clojars         4
# alire           1



pkgman
freebsd12     381
conan         341
vcpkg         337
openbsd69     271
netbsd9       180
homebrew       88
chromebrew     86
npm            27
cargo          26
clojars         4
alire           1
dtype: int64

In [22]:
def get_pkgmans(pkgman):
    pkgs = nu_df[nu_df['pkgman'] == pkgman]['pkg_name'].to_list()
    unique_pkgmans = nu_df[nu_df['pkg_name'].isin(pkgs)]['pkgman'].unique()
    unique_pkgmans_list = unique_pkgmans.tolist()
    unique_pkgmans_list.remove(pkgman)
    unique_pkgmans_list.sort()
    return unique_pkgmans_list

print(get_pkgmans('freebsd12'))
print(get_pkgmans('conan'))
print(get_pkgmans('vcpkg'))
print(get_pkgmans('openbsd69'))
print(get_pkgmans('netbsd9'))
print(get_pkgmans('homebrew'))
print(get_pkgmans('chromebrew'))
print(get_pkgmans('npm'))
print(get_pkgmans('cargo'))
print(get_pkgmans('clojars'))
print(get_pkgmans('alire'))


['netbsd9', 'openbsd69']
['cargo', 'chromebrew', 'homebrew', 'npm', 'vcpkg']
['alire', 'cargo', 'chromebrew', 'conan', 'homebrew', 'npm']
['freebsd12', 'netbsd9']
['freebsd12', 'openbsd69']
['chromebrew', 'conan', 'npm', 'vcpkg']
['conan', 'homebrew', 'vcpkg']
['cargo', 'clojars', 'conan', 'homebrew', 'vcpkg']
['conan', 'npm', 'vcpkg']
['npm']
['vcpkg']


In [34]:
nu_df['pkg_name'].mode()

0    brotli
1      grpc
dtype: object

In [45]:
def get_number_of_deps(pkgman, pkgname):
    ddf = pd.read_csv(f"../../dasea_data/{pkgman}/{pkgman}_dependencies_05-17-2022.csv")
    ddf = ddf[ddf['target_name'] == pkgname]
    return len(ddf)

In [11]:
data_path = '../../dasea_data'
path_dict = {
    'alire' : '/alire/alire_dependencies_05-17-2022.csv',
    'cargo' : '/cargo/cargo_dependencies_05-17-2022.csv',
    'chromebrew' : '/chromebrew/chromebrew_dependencies_05-17-2022.csv',
    'clojars' : '/clojars/clojars_dependencies_05-19-2022.csv',
    'conan' : '/conan/conan_dependencies_05-17-2022.csv',
    'freebsd12' : '/ports/freebsd12/freebsd12_dependencies_05-17-2022.csv',
    'homebrew' : '/homebrew/homebrew_dependencies_05-17-2022.csv',
    'netbsd9' : '/ports/netbsd9/netbsd9_dependencies_05-17-2022.csv',
    'openbsd69' : '/ports/openbsd69/openbsd69_dependencies_05-17-2022.csv',
    'vcpkg' : '/vcpkg/vcpkg_dependencies_05-17-2022.csv',
    'npm' : '/npm/npm_dependencies_05-17-2022.csv'
}

df_dict = {}

def _df(path):
    import os
    file_path = os.path.abspath(data_path + path)
    return pd.read_csv(file_path)


outfil = pd.read_csv('out/non_unique_repo.csv')

all_pkgs = outfil['pkg_name'].unique()
res_df = pd.DataFrame(columns=('pkgname','ecosystem', 'count_deps', 'count_total'))
counter = 0

for pkg in all_pkgs:
    counter += 1
    print(counter)
    list_of_pkgmans = outfil[outfil['pkg_name'] == pkg]['pkgman'].to_list()
    acc_list = []
    acc = 0
    for pman in list_of_pkgmans:
        # if pman != 'npm': # dirty fix - remove when ready for real data
        path = path_dict[pman]
        ddf
        if path in df_dict:
            ddf = df_dict[path]
        else:
            ddf = _df(data_path + path)
            df_dict[path] = ddf
        acc = acc + len(ddf[ddf['target_name'] == pkg])
        acc_list.append(len(ddf[ddf['target_name'] == pkg]))
        # else:
        #     acc_list.append(-1)
    
    

    entry = pd.DataFrame.from_dict({
        "pkgname": [pkg],
        "ecosystem":  [list_of_pkgmans],
        "count_deps": [acc_list],
        "count_total": [acc]
    })
    res_df = pd.concat([res_df, entry], ignore_index=True)





# vdf = pd.read_csv(f"../../dasea_data/vcpkg/vcpkg_dependencies_05-17-2022.csv")
# cdf = pd.read_csv(f"../../dasea_data/conan/conan_dependencies_05-17-2022.csv")

# lenv = len(vdf[vdf['target_name']== 'bullet3'])
# lenc = len(cdf[cdf['target_name']== 'bullet3'])

# lenv+lenc


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
res_df.sort_values(by=['count_total'], ascending=False).to_csv('out/res_df.csv')